# Model based analysis of Risk and Ambiguity

Using the utility function to try and asses Risk and ambiguity aversion using the formula from 

SV = (probability − β (Ambiguity / 2)] × Value ** α 


In [1]:
import pandas as pd
import numpy as np

import pymc3 as pm
import arviz as az

import matplotlib.pyplot as plt

load monatery data add subject sequential number starting from 0

In [2]:
db = pd.read_csv('mon.csv')
db['subn'] = db['sub'].rank(method='dense').astype(int)-1
db.head()

,Unnamed: 0,choice,value,risk,ambiguity,sub,catch,subn
0,0,0.0,5,0.75,0.00,11,0.0,1
1,1,0.0,8,0.50,0.24,11,0.0,1
2,2,0.0,25,0.50,0.74,11,0.0,1
3,3,0.0,25,0.50,0.74,11,0.0,1
4,4,0.0,25,0.25,0.00,11,0.0,1


transfer values and subjects to arrays

In [3]:
sub_idx = np.arange(51)
value = np.asarray(db['value'])
n_subs = max(sub_idx) + 1
n_trials = 84
 # index for trials as well
trials = np.arange(n_trials)
n_subs

51

In [4]:
sub_idx

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [11]:
# need to create matrices
valueMat = np.reshape(value, (n_subs, n_trials)).T
choiceMat = np.reshape(db.choice.values, (n_subs, n_trials)).T
riskMat = np.reshape(db.risk.values, (n_subs, n_trials)).T
ambMat = np.reshape(db.ambiguity.values, (n_subs, n_trials)).T

### model 1: risk aversion without ambiguity

In [9]:
 with pm.Model() as Risk:

    # Priors for unknown model parameters
   # intercept = pm.Normal("intercept", mu=0, sigma=10)
    rMu = pm.Normal('rMu', 0,1)
    rSig = pm.Exponential('rSig', 1)
    nMu = pm.Normal('nMu', 0,1)
    nSig = pm.Exponential('nSig', 1)
    α = pm.Lognormal('α', rMu, rSig,  shape = n_subs)
    γ = pm.Lognormal('γ',nMu , nSig, shape = n_subs)
    
    sigma = pm.HalfNormal("sigma", sigma=1)

    # Expected value of outcome
    svLotto = (valueMat**α[sub_idx]) * riskMat
    svRef = 5**α[sub_idx]
    mu =  1/( 1 + np.exp(-(svLotto - svRef)/γ[sub_idx]))
    
    #y_hat = 1/(1+np.exp(γ * (safe-bet))) + eps
    
    # Likelihood (sampling distribution) of observations
   # Y_obs = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=choiceMat)
    choice = pm.Binomial('choice',1, mu, observed=choiceMat)
    trace = pm.sample(1000, return_inferencedata=True, nuts={'target_accept':0.9})

/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/pymc3/model.py:1755: ImputationWarning: Data in choice contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>NUTS: [sigma, γ, α, nSig, nMu, rSig, rMu]
>Metropolis: [choice_missing]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 31 seconds.
/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
There were 45 divergences after tuning. Increase `target_accept` or reparameterize.
There were 52 divergences after tuning. Increase `target_accept` or reparameterize.
There were 32 divergences after tuning. Increase `target_accept` or reparameterize.
There were 33 divergences after tuning. Increase `target_accept` or reparameterize.


In [10]:
az.summary(trace, var_names='α')

Got error No model on context stack. trying to find log_likelihood in translation.
/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/arviz/data/io_pymc3_3x.py:102: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
α[0],0.149,0.028,0.094,0.200,0.000,0.000,4693.0,2202.0,1.0
α[1],0.742,0.059,0.634,0.856,0.001,0.001,5344.0,2934.0,1.0
α[2],0.839,0.040,0.761,0.916,0.001,0.000,4244.0,2435.0,1.0
α[3],0.801,0.037,0.727,0.869,0.001,0.000,3966.0,2270.0,1.0
α[4],0.572,0.164,0.242,0.855,0.003,0.002,3629.0,2428.0,1.0
α[5],0.774,0.100,0.589,0.966,0.002,0.001,4098.0,2510.0,1.0
α[6],0.548,0.040,0.472,0.621,0.001,0.000,5090.0,3003.0,1.0
α[7],0.798,0.099,0.614,0.986,0.002,0.001,4052.0,2208.0,1.0
α[8],0.713,0.060,0.603,0.828,0.001,0.001,5186.0,2932.0,1.0
α[9],0.279,0.043,0.195,0.354,0.001,0.000,4765.0,2505.0,1.0


### model 2: risk and ambiguity

In [16]:
with pm.Model() as RiskAmb:
    # hyper
    rMu = pm.Normal('rMu', 0,1)
    rSig = pm.Exponential('rSig', 1)
    aMu = pm.Normal('aMu', 0,1)
    aSig = pm.Exponential('aSig', 1)
    
    nMu = pm.Normal('nMu', 0,1)
    nSig = pm.Exponential('nSig', 1)
    
    
    α = pm.Lognormal('α', rMu, rSig,  shape = n_subs)
    γ = pm.Lognormal('γ',nMu , nSig, shape = n_subs)
    # Priors for unknown model parameters
    
    β = pm.Lognormal('β', aMu, aSig, shape = n_subs)
    
    
    sigma = pm.HalfNormal("sigma", sigma=1)

    # Expected value of outcome
    svLotto = valueMat ** α[sub_idx] * -(β[sub_idx] * (ambMat/2) - riskMat)
    svRef = 5 ** α[sub_idx]
    mu =  1/( 1 + np.exp(-(svLotto - svRef)/γ[sub_idx]))
       
    # Likelihood (sampling distribution) of observations
    #Y_obs = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=db.choice)
    choice = pm.Binomial('choice',1, mu, observed=choiceMat)
    trace = pm.sample(1000, return_inferencedata=True, nuts={'target_accept':0.9})

/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/pymc3/model.py:1755: ImputationWarning: Data in choice contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>NUTS: [sigma, β, γ, α, nSig, nMu, aSig, aMu, rSig, rMu]
>Metropolis: [choice_missing]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 76 seconds.
/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
There were 26 divergences after tuning. Increase `target_accept` or reparameterize.
There were 16 divergences after tuning. Increase `target_accept` or reparameterize.
There were 14 divergences after tuning. Increase `target_accept` or reparameterize.
There were 41 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


In [19]:
az.summary(trace, var_names=['α','β'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
α[0],0.156,0.030,0.097,0.210,0.001,0.000,3470.0,2036.0,1.0
α[1],0.865,0.066,0.735,0.983,0.001,0.001,2650.0,2537.0,1.0
α[2],0.977,0.064,0.867,1.089,0.001,0.001,2608.0,2653.0,1.0
α[3],0.849,0.040,0.770,0.922,0.001,0.001,3095.0,2480.0,1.0
α[4],0.632,0.183,0.273,0.942,0.003,0.002,2972.0,2696.0,1.0
...,...,...,...,...,...,...,...,...,...
β[46],0.689,0.102,0.509,0.899,0.002,0.001,2449.0,1721.0,1.0
β[47],0.188,0.114,0.009,0.395,0.002,0.002,2332.0,2214.0,1.0
β[48],1.012,0.148,0.753,1.284,0.003,0.002,2096.0,982.0,1.0
β[49],0.273,0.167,0.011,0.572,0.003,0.002,1953.0,1834.0,1.0
